In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Gram-Schmidt Orthogonalization 

In [2]:
def proj(u: np.array, v: np.array, eps=0.0001) -> np.array:
    """projection of the vector v onto the vector u"""

    norm_sq = np.dot(u,u)
    if norm_sq < eps:
        return np.zeros(3, dtype=np.float64)
    else:
        return (np.dot(u,v)/np.dot(u,u))*u


def gram_schmidt(basis: list, eps=0.0001) ->list:
    """return an orthogonal basis set via the gram-schmidt process"""

    #normalize basis
    for i, v in enumerate(basis):
        basis[i] = basis[i].astype(np.float64)
        basis[i] /= np.sqrt(np.dot(v, v))

    #gram-schmidt algorithm
    ortho_basis = []
    for i, v in enumerate(basis):
        ortho_basis.append(v)
        for j in range(i):
            ortho_basis[i] -= proj(basis[j], v, eps=eps)

    #remove zero vectors and normalize orthogonal basis
    zero_indices = []
    for i, v in enumerate(ortho_basis):
        norm_sq = np.dot(v,v)
        if norm_sq < eps:
            zero_indices.append(i)
        else:
            ortho_basis[i] /= np.sqrt(norm_sq)
    for i in zero_indices:
        ortho_basis.pop(i)

    return ortho_basis

Orthogonal Matrices in 3-Dimensions

In [3]:
def rotation(axis: np.array, theta: float) -> np.array:
    """
    return a rotation matrix about the given axis by the given angle, theta

    Parameters
    ----------
    axis: np.array
        np.array with shape (3,) which represents the axis of rotation
    theta: float
        rotation angle

    Returns 
    -------
    rot_matrix: np.array
        rotation matrix with shape (3, 3)
    """
    #Ensure that the axis of rotation is normalized
    axis = np.array(axis, dtype=np.float64)
    axis /= np.sqrt(np.dot(axis, axis))
    
    s = np.sin(theta)
    c = np.cos(theta)

    rot_matrix = np.array([[c + (axis[0]**2)*(1-c),             axis[0]*axis[1]*(1-c) - axis[2]*s,  axis[0]*axis[2]*(1-c) + axis[1]*s],
                           [axis[0]*axis[1]*(1-c) + axis[2]*s,  c + (axis[1]**2)*(1-c),             axis[1]*axis[2]*(1-c) - axis[0]*s],
                           [axis[0]*axis[2]*(1-c) - axis[1]*s,  axis[1]*axis[2]*(1-c) + axis[0]*s,  c + (axis[2]**2)*(1-c)           ]])
    return rot_matrix

def reflection(normal: np.array) -> np.array:
    """
    return a reflection matrix across a plane normal to the given vector

    Parameters
    ----------
    normal: np.array
        np.array with shape (3,) which represents the axis of rotation

    Returns 
    -------
    ref_matrix: np.array
        reflection matrix with shape (3, 3)
    """
    #normalize normal vector
    normal = np.array(normal, dtype=np.float64)
    normal /= np.sqrt(np.dot(normal, normal))

    #construct an orthonormal basis which includes the normal vector
    basis = [normal, np.array([1.,0.,0.]), np.array([0.,1.,0.]), np.array([0.,0.,1.])]
    ortho_basis = gram_schmidt(basis)
    P = np.vstack(ortho_basis)
    diag = np.array([[-1., 0., 0.],
                     [ 0., 1., 0.],
                     [ 0., 0., 1.]])
    ref_matrix = P.T@diag@P
    return ref_matrix


Affine Transformation Matrices in 3-Dimensions

In [4]:
def affine(ortho: np.array, trans: np.array) -> np.array:
    """
    create a 3D affine transformation matrix given an orthogonal transformation matrix and a translation vector

    Parameters
    ----------
    ortho: np.array
        orthogonal transformation matrix in the form of an np.array with shape (3,3) 

    trans:  np.array
        translation vector in the form of an np.array with shape (3,)

    Returns 
    -------
    affine_matrix: np.array
        affine transformation matrix with shape (4, 4)
    """
    affine_matrix = np.zeros((4,4), dtype=np.float64)
    affine_matrix[:3, :3] = ortho
    affine_matrix[:-1,3] = np.array(trans, dtype=np.float64)
    affine_matrix[3,3] = 1.
    return affine_matrix
    

Octahedral Symmetry Group, $O_h$

The operations which compose the octahedral point group, $O_h$, of an octahedron with vertices at integer positions nearest to the origin as follows

$$(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)$$

Will be stored in a dictionary for later use.

In [5]:
# A dictionary containing the 3x3 orthogonal matrices which represent the symmetry operations of the octahedral symmetry group
octahedron = {

#Identity
"E" : np.eye(3),

#Inversion
"i"  : -np.eye(3),

#120° rotation through the normal of each of the triangular faces of the octahedron
"C_1_3_1" : rotation([ 1, 1, 1], 2*np.pi/3),
"C_1_3_2" : rotation([-1, 1, 1], 2*np.pi/3),
"C_1_3_3" : rotation([-1,-1, 1], 2*np.pi/3),
"C_1_3_4" : rotation([ 1,-1, 1], 2*np.pi/3),

#240° rotation through the normal of each of the triangular faces of the octahedron
"C_2_3_1" : rotation([ 1, 1, 1], 4*np.pi/3),
"C_2_3_2" : rotation([-1, 1, 1], 4*np.pi/3),
"C_2_3_3" : rotation([-1,-1, 1], 4*np.pi/3),
"C_2_3_4" : rotation([ 1,-1, 1], 4*np.pi/3),

#180° rotation thorugh the each of the edges
"C_2_1" : rotation([ 1, 1, 0], np.pi),
"C_2_2" : rotation([-1, 1, 0], np.pi),
"C_2_3" : rotation([ 1, 0, 1], np.pi),
"C_2_4" : rotation([ 0, 1, 1], np.pi),
"C_2_5" : rotation([-1, 0, 1], np.pi),
"C_2_6" : rotation([ 0,-1, 1], np.pi),

#mirror reflections about the xy, xz, and yz planes
"sigma_h_xy" : reflection([0, 0, 1]),
"sigma_h_xz" : reflection([0, 1, 0]),
"sigma_h_xy" : reflection([1, 0, 0]),

#mirror reflections which do not contain any edges
"sigma_d_1" : reflection([ 1, 1, 0]),
"sigma_d_2" : reflection([-1, 1, 0]),
"sigma_d_3" : reflection([ 1, 0, 1]),
"sigma_d_4" : reflection([-1, 0, 1]),
"sigma_d_5" : reflection([ 0, 1, 1]),
"sigma_d_6" : reflection([ 0,-1, 1]),

#60° roto-reflection about triangular face normals
"S_6_1" : rotation([ 1, 1, 1], np.pi/3)@reflection([ 1, 1, 1]),
"S_6_2" : rotation([-1, 1, 1], np.pi/3)@reflection([-1, 1, 1]),
"S_6_3" : rotation([-1,-1, 1], np.pi/3)@reflection([-1,-1, 1]),
"S_6_4" : rotation([ 1,-1, 1], np.pi/3)@reflection([ 1,-1, 1]),

#90° roto-reflection about x, y, and, z axes
"S_4_1" : rotation([1,0,0], np.pi)@reflection([1,0,0]),
"S_4_2" : rotation([0,1,0], np.pi)@reflection([0,1,0]),
"S_4_3" : rotation([0,0,1], np.pi)@reflection([0,0,1])

}

In [6]:
def array_equal(a1: np.array, a2: np.array, eps=0.0001) -> np.array:
    """
    Check if two arrays are equal up to a accuracy factor eps

    Parameters
    ----------
    a1 : np.array
        the first array

    a2 : np.array
        the second array

    eps: float
        check that each element of the arrays differ by no more than eps

    Return
    ------
    answer : bool
        returns true if the arrays are equal
    """
    diff = np.abs(a1 - a2)
    return np.all(diff < eps)

In [10]:
#Check the closure of the group

group_elements = list(octahedron.values())

n = len(group_elements)
for i in range(n):
    for j in range(n):
        product = group_elements[i] @ group_elements[j]
        for k in range(n):
            if array_equal(product, group_elements[k]):
                print(i, j)
                break
            

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
1 0
1 1
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
2 0
2 2
2 4
2 6
2 7
2 9
2 11
2 14
2 15
2 17
2 19
2 21
2 23
2 24
2 25
2 26
3 0
3 3
3 5
3 6
3 7
3 8
3 10
3 12
3 15
3 16
3 18
3 20
3 23
3 25
3 26
3 27
4 0
4 2
4 4
4 7
4 8
4 9
4 11
4 12
4 13
4 17
4 19
4 20
4 22
4 24
4 26
4 27
5 0
5 3
5 5
5 6
5 8
5 9
5 10
5 13
5 14
5 16
5 18
5 21
5 22
5 24
5 25
5 27
6 0
6 1
6 2
6 3
6 5
6 6
6 8
6 11
6 14
6 15
6 17
6 19
6 21
6 23
6 25
6 26
6 28
6 29
6 30
7 0
7 1
7 2
7 3
7 4
7 7
7 9
7 10
7 12
7 15
7 16
7 18
7 20
7 23
7 26
7 27
7 28
7 29
7 30
8 0
8 1
8 3
8 4
8 5
8 6
8 8
8 11
8 12
8 13
8 17
8 19
8 20
8 22
8 24
8 27
8 28
8 29
8 30
9 0
9 1
9 2
9 4
9 5
9 7
9 9
9 10
9 13
9 14
9 16
9 18
9 21
9 22
9 24
9 25
9 28
9 29
9 30
10 0
10 1
10 3
10 5
10 7
10 9
10 10
10 12
10 13
10 14
10 15
10 18
10 22
10 23
10 25
10 2

Tetrahedral Symmetry Group, $T_d$

In [8]:
# A dictionary containing the 3x3 orthogonal matrices which represent the symmetry operations of the octahedral symmetry group
tetrahedron = {

#Identity
"E" : np.eye(3),


}